In [1]:
print("welcome to data analysis.")

welcome to data analysis.


In [2]:
run ./scripts/analysis_scripts.py

In [3]:
minimum_path = './computational-results-minimum/results_datatable'
additive_faces_path = './computational-results-additive-faces/results_datatable'
objective_zero_path = './computational-results-objective-zero/results_datatable'
objective_one_percent_path = './computational-results-objective-one-percent/results_datatable/'

In [8]:
path = objective_one_percent_path
df = generate_dataframe(path,stats='mean_t_distribution')

In [9]:
shifted_gmean_df(df, s=1)

,metric,sbb_bb_constant_bound,sbb_bfs_mixed_bound,sbb_bfs_constant_bound,sbb_dfs_lp_bound,sbb_bb_lp_bound,sbb_dfs_mixed_bound,sbb_bfs_lp_bound,naive_computation,sbb_dfs_constant_bound,sbb_bb_mixed_bound
0,shifted_geometric_mean(shift=1),3.133346,4.713257,3.053098,7.324454,7.238484,4.567075,7.192693,9.992228,2.968973,4.584099
